# Import related pacakages

In [1]:
import cobra
import sys
sys.path.append(r'./code/')
from cobrapy_ec_model_function import *

# Input and output data

In [2]:
#enyme model needed data
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
sigma = 1 #kapp data sigma is 1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

model_name = './model/iML1515_new.xml' 
c13reaction_file = './data/C13reaction.csv' 

#max kcat for EC number selected from BRENDA and SABIO-RK database(use autoPACMEN)
kcat_database_combined_file= './data/Brenda_sabio_combined_select.json'

reaction_kcat_MW_file = './analysis/reaction_kcat_MW.csv'
fluxes_infile_ori = './analysis/ECMpy_ori_solution_df_pfba.csv'
json_model_path = './model/iML1515_irr_enz_constraint.json'

enz_ratio=0.01
reaction_enz_usage_file = './analysis/ECMpy_adj_round1_reaction_enz_usage_df.csv'
reaction_kcat_MW_round1_outfile = './analysis/reaction_change_by_enzuse.csv'
json_round1_output_file = './model/iML1515_irr_enz_constraint_adj_round1.json'
round1_fluxes_outfile = './analysis/ECMpy_adj_round1_solution_df_pfba.csv'

c13_percentage=0.1
json_round2_output_file= './model/iML1515_irr_enz_constraint_adj_round2.json'
reaction_kcat_MW_round2_outfile = './analysis/reaction_change_by_c13.csv'
round2_fluxes_outfile = './analysis/ECMpy_adj_round2_solution_df_pfba.csv'

# Step1:  Calibration kcat according Enzyme usage 

In [3]:
enz_model=get_enz_model_use_enz_usage_by_eckcat(enz_ratio,json_model_path,fluxes_infile_ori,reaction_kcat_MW_file,\
                                      reaction_enz_usage_file,kcat_database_combined_file, model_name, \
                                      f, ptot, sigma, lowerbound, upperbound, json_round1_output_file, \
                                      reaction_kcat_MW_round1_outfile)

enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,round1_fluxes_outfile,reaction_kcat_MW_round1_outfile)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

c13reaction_2_enz_model_diff = get_diff_reaction_use_c13(c13reaction_file,enz_model_pfba_solution)
print (c13reaction_2_enz_model_diff)

need changing reaction: 
['GAPD', 'ENO', 'FBA_num1', 'NADH16pp', 'RBFSa', 'KARA2', 'KARA1_reverse', 'ACCOAC', '5DKGR_num1', 'METS_num2', 'PGK_reverse', 'GND', 'PFL_num3', 'PGM_reverse_num2', 'GHMT2r', 'GLNS', 'PPC', 'METAT', 'PGI', 'ATPS4rpp_num2']
changed reaction: 
['GAPD', 'ENO', 'FBA_num1', 'NADH16pp', 'RBFSa', 'KARA2', 'ACCOAC', 'METS_num2', 'GND', 'GHMT2r', 'GLNS', 'PPC', 'METAT', 'PGI']
0.5594463187828271
['ICL', 'AKGDH', 'SUCDi', 'ME2', 'SUCOAS_reverse', 'PDH']


# Step2: Calibration kcat according c13 reaction list

In [4]:
enz_model=get_enz_model_use_c13(reaction_kcat_MW_round1_outfile, json_model_path, c13reaction_file, c13_percentage, \
                                kcat_database_combined_file,model_name, f, ptot, sigma, lowerbound, \
                                upperbound, json_round2_output_file,reaction_kcat_MW_round2_outfile)

enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,round2_fluxes_outfile,reaction_kcat_MW_round2_outfile)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

c13reaction_2_enz_model_diff = get_diff_reaction_use_c13(c13reaction_file,enz_model_pfba_solution)
print (c13reaction_2_enz_model_diff)

need changing reaction: 
['PDH', 'AKGDH']
changed reaction: 
['PDH', 'AKGDH']
0.6802441208858396
['ME2', 'ICL', 'SUCOAS_reverse']


# Step3: Solveing enzyme concentration constraint by COBRApy.

In [5]:
#run enzyme constraint metabolic model
enz_model=get_enzyme_constraint_model(json_round2_output_file)
#enz_model.reactions.get_by_id('EX_glc__D_e_reverse').bounds = (13, 13)
pfba_solution = cobra.flux_analysis.pfba(enz_model)
pfba_solution_df = pfba_solution.to_frame()
pfba_solution_df.to_csv('./analysis/ECMpy_solution_df_pfba.csv')
print(pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])
print(pfba_solution.fluxes['EX_ac_e'])

0.6802441208858396
7.711579625177092


In [6]:
#run genome-scale metabolic model
norm_model=cobra.io.json.load_json_model(json_round2_output_file)
#norm_model.reactions.get_by_id('EX_glc__D_e_reverse').bounds = (13, 13)
pfba_solution = cobra.flux_analysis.pfba(norm_model)
pfba_solution_df = pfba_solution.to_frame()
#pfba_solution_df.to_csv('./analysis/Orimodel_solution_df_pfba.csv')
print(pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])
print(pfba_solution.fluxes['EX_ac_e'])

0.8697726420320147
0.0
